In [1]:
import cPickle as pickle
import os, sys, random
sys.path.append('/Users/matt.meng/dev/seq2seq_model')

In [2]:
import pandas as pd
from data_preprocess import tokenize_title_column, convert_text_JSON_to_csv, process_title_column, \
process_title_column_by_spacy, create_selected_vocab_dict, process_title_with_token_dict, create_crambled_training

### the title processing steps

In [3]:
data_path = '/Users/matt.meng/Downloads'
file_name = 'insights_selected_articles_20170818_20170926.json'
meta_data_file_name = 'meta_title_data.csv'
delimiter = '\t\t'

In [4]:
'''
convert_text_JSON_to_csv(os.path.join(data_path, file_name),
                         os.path.join(data_path, meta_data_file_name),
                         delimiter)
'''

'\nconvert_text_JSON_to_csv(os.path.join(data_path, file_name),\n                         os.path.join(data_path, meta_data_file_name),\n                         delimiter)\n'

In [5]:
data = pd.read_csv(os.path.join(data_path, meta_data_file_name), index_col='url', delimiter=delimiter, encoding='utf-8')


/Users/matt.meng/.virtualenvs/kaggle/lib/python2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [6]:
print data.shape
data.head()

(984000, 3)


,title,traffic,publisherId
url,,,
http://www.kgw.com/news/local/rescue-effort-underway-for-hikers-trapped-by-eagle-creek-fire/470354816,Rescue effort underway for hikers trapped by E...,636961,1037860
https://smartasset.com/retirement/the-top-11-cities-for-a-worktirement-2017-edition,The Top 11 Cities for a Worktirement - 2017 Ed...,4967,1040526
http://www.complex.com/pop-culture/2017/07/chrissy-teigen-response-kim-kardashian-cocaine-controversy,Chrissy Teigen Had an Incredible Response to t...,154611,1042924
http://www.womansworld.com/posts/recreated-photos-102484/photos/recreated-photos-5-157859,17 Photos of Recreated Family Moments That'll ...,17784,1075445
http://www.cbs.com/shows/bull/video/XdBBwmEq__oWkKnzq7IHnaPYl_bkr9X7/bull-it-s-classified/,Bull Video - It's Classified,1005,1041535


In [7]:
data.sort_values(['traffic'], ascending=False, inplace=True)

In [8]:
data.head()

,title,traffic,publisherId
url,,,
https://weather.com/storms/hurricane/news/hurricane-irma-bahamas-florida-georgia-carolinas-forecast,Hurricane Irma on Its Way to the Bahamas as a ...,11508824,1187
https://weather.com/storms/hurricane/news/hurricane-irma-united-states-hurricane-warning-puerto-rico-leeward-islands-0,Hurricane Irma on Its Way to the Bahamas as a ...,9787772,1187
http://newarena.com/nfl/ex-nfl-player-who-wanted-son-dead-17-years-ago-is-in-for-a-shocking-surprise/,Ex-NFL Player Who Wanted Son Dead 17 Years Ago...,9016715,1023235
https://weather.com/storms/hurricane/news/hurricane-harvey-forecast-gulf-coast-texas-louisiana,Harvey's Real Story Begins: 40 Inches of Rain ...,5055129,1187
https://weather.com/storms/hurricane/news/tropical-storm-hurricane-jose-forecast-atlantic,Tropical Storm Jose Strengthens in the Atlanti...,4401042,1187


#### check the duplicates

In [9]:
sum(data.index.duplicated())

44966

In [10]:
unique_data = data[~data.index.duplicated(keep='first')]

In [11]:
print unique_data.shape

(939034, 3)


####  tokenize the titles, remove non-Enligh URL and titles

In [12]:
processed_column_name = 'processed_title'
pageView_column_name = 'traffic'
filtered_data = tokenize_title_column(unique_data, processed_column_name, pageView_column_name)

/Users/matt.meng/dev/seq2seq_model/data_preprocess.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['title_word_counts'], data[processed_column_name] = zip(*data[title_column_name].map(basic_tokenizer))


finish the tokenization...


In [13]:
print filtered_data.shape
filtered_data.head()

(908334, 5)


,title,traffic,publisherId,title_word_counts,processed_title
url,,,,,
http://www.msn.com/en-us/news/breakingnews/live-coverage-from-cbs-news/ar-BBmYvYY,Live Coverage from CBS News,3174981,1023406,5,live coverage from cbs news
http://clark.com/personal-finance-credit/credit-freeze-and-thaw-guide/,Credit freeze and thaw guide,1555017,1016433,5,credit freeze and thaw guide
http://www.nbcmiami.com/news/local/Watch-Live-NBC-6-News-204316851.html,WATCH LIVE: NBC 6 News,811635,1030944,5,watch live nbc ## news
https://weather.com/tv/the-weather-channel-live/video/watch-the-weather-channel-live,Watch The Weather Channel Live,569573,1187,5,watch the weather channel live
http://www.msn.com/en-us/news/video/live-news-coverage-from-cbs-news/ar-BBmYvYY,Live Coverage from CBS News,547493,1023406,5,live coverage from cbs news


In [14]:
titile_dict = {}
for title in filtered_data['processed_title']:
    titile_dict[title] = titile_dict.get(title, 0) + 1

In [15]:
sorted_titles = sorted(titile_dict.items(), key=lambda x: x[1], reverse=True)
print len(sorted_titles)

598154


In [16]:
sorted_titles[:100]

[('former tulsa principal pleads guilty to child porn charge news## com',
  2055),
 ('espn updates fpi rankings after week ##', 510),
 ('big brother video episode ## cbs com', 389),
 ('## things to know for today', 136),
 ('ap top entertainment news at ## ## p m edt', 128),
 ('week ## college football bowl projections', 119),
 ('ap top international news at ## ## p m edt', 109),
 ('samsung direct from them note ## pre order thread page ## android forums at androidcentral com',
  99),
 ('week ## amway coaches poll released', 99),
 ('see the entire kennedy family', 99),
 ('steely dan co founder guitarist walter becker dies at ##', 96),
 ('floyd mayweather vs conor mcgregor lives up to billing', 95),
 ('t mobile galaxy note ## pre order thread page ## android forums at androidcentral com',
  95),
 ('fox ## phoenix breaking news local headlines weather traffic and more ksaz',
  93),
 ('the girlfriends of college football stars', 92),
 ('palace announces prince william kate expecting third 

In [17]:
processed_column_name = 'processed_title'
pageView_column_name = 'traffic'

all_titles, vocab_dict = process_title_column(filtered_data, processed_column_name, pageView_column_name)


total 86744 words are tokenized from 598075 titles using 7.83 second


In [18]:
'''
data['publisherId'] = data['publisherId'].astype(int).astype(str)
valid_publisher_ids = ['1001082', '1023406', '1003264', '1040522', '782', '1006541',
                           '1168', '1038583', '1021516', '580', '1020689', '1031851', '1001264',
                           '1039208', '1054980', '1018671', '1031841', '1031842', '1031852',
                           '1008941', '1003764', '1068057', '1038711', '1002628', '1031853',
                           '1021578', '1043813', '1010748', '1040526', '1005092', '612',
                           '1003870', '1001156', '1012083', '1017946', '1041479', '1027016',
                           '1010488', '1017947', '1010497', '1038582', '1045821', '1020968',
                           '1037842', '1029984', '723', '196', '1030941']

filtered_data = data.loc[data['publisherId'].isin(valid_publisher_ids), :]
unique_filtered_data = filtered_data[~filtered_data.index.duplicated(keep='first')]
print unique_filtered_data.shape

processed_column_name = 'processed_title'
pageView_column_name = 'traffic'
all_titles, vocab_dict = process_title_column_by_spacy(filtered_data, 'processed_title', 'traffic', skip_stop_words=True)
'''


"\ndata['publisherId'] = data['publisherId'].astype(int).astype(str)\nvalid_publisher_ids = ['1001082', '1023406', '1003264', '1040522', '782', '1006541',\n                           '1168', '1038583', '1021516', '580', '1020689', '1031851', '1001264',\n                           '1039208', '1054980', '1018671', '1031841', '1031842', '1031852',\n                           '1008941', '1003764', '1068057', '1038711', '1002628', '1031853',\n                           '1021578', '1043813', '1010748', '1040526', '1005092', '612',\n                           '1003870', '1001156', '1012083', '1017946', '1041479', '1027016',\n                           '1010488', '1017947', '1010497', '1038582', '1045821', '1020968',\n                           '1037842', '1029984', '723', '196', '1030941']\n\nfiltered_data = data.loc[data['publisherId'].isin(valid_publisher_ids), :]\nunique_filtered_data = filtered_data[~filtered_data.index.duplicated(keep='first')]\nprint unique_filtered_data.shape\n\nproces

#### check the `vocab_ditc` content

In [19]:
sorted_pairs = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)

In [20]:
sorted_pairs

[('##', 168226),
 ('to', 155725),
 ('in', 138767),
 ('the', 116565),
 ('of', 90630),
 ('for', 83362),
 ('on', 57583),
 ('and', 47833),
 ('with', 41922),
 ('at', 40073),
 ('is', 35256),
 ('after', 31922),
 ('new', 27078),
 ('from', 25554),
 ('man', 24075),
 ('trump', 24008),
 ('you', 21867),
 ('not', 21737),
 ('how', 20181),
 ('police', 20026),
 ('hurricane', 17913),
 ('are', 17753),
 ('as', 17613),
 ('by', 16473),
 ('this', 15709),
 ('will', 14760),
 ('it', 14667),
 ('your', 14474),
 ('about', 14196),
 ('irma', 14041),
 ('what', 13792),
 ('be', 13638),
 ('state', 13483),
 ('out', 12911),
 ('that', 12833),
 ('football', 12822),
 ('up', 12730),
 ('harvey', 12548),
 ('over', 12350),
 ('says', 11862),
 ('year', 11762),
 ('woman', 11493),
 ('school', 11198),
 ('video', 11072),
 ('who', 10551),
 ('his', 10511),
 ('more', 10007),
 ('county', 9876),
 ('home', 9656),
 ('game', 9497),
 ('watch', 9430),
 ('old', 9178),
 ('has', 9124),
 ('why', 8970),
 ('have', 8898),
 ('first', 8872),
 ('no', 876

### create the `token_dict` and the scrambled data

In [21]:
_PAD = b"_PAD"
_GO = b"_GO"
_EOS = b"_EOS"
_UNK = b"_UNK"
_START_VOCAB = [_PAD, _GO, _EOS, _UNK]

TOKEN_DICT = {}
REVERSE_TOKEN_DICT = {}
for i in xrange(len(_START_VOCAB)):
    TOKEN_DICT[_START_VOCAB[i]] = i
    REVERSE_TOKEN_DICT[i] = _START_VOCAB[i]

In [22]:
UKN_index = len(TOKEN_DICT) - 1
token_dict, reverse_token_dict = create_selected_vocab_dict(vocab_dict, UKN_index, token_freq_threshold=4)
selected_content = process_title_with_token_dict(all_titles, token_dict, reverse_token_dict, UKN_index, UKN_frac_threshold=0.2)
processed_content = create_crambled_training(selected_content)

total 38607 unique tokens are included in the token dictionary...
total 566819 titles are included...
finish generating scrambled titles, ignore 0 titles..


### load the processed pickle and check the content

In [ ]:
data_path = '/Users/matt.meng'
#pickle_file = 'processed_titles_data.pkl'
pickle_file = 'lemmanized_no_stop_words_CBOW_data.pkl'
with open(os.path.join(data_path, pickle_file), 'r') as input_file:
     data = pickle.load(input_file)

In [ ]:
data.keys()

In [ ]:
training_list = data['training_list']
target_list = data['target_list']

In [ ]:
print len(training_list)

In [ ]:
training_list[:4]

In [ ]:
titles = data['titles']
token_dict = data['token_dict']
reverse_token_dict = data['reverse_token_dict']

In [ ]:
token_dict.keys()[:20]

In [ ]:
index = 50
map(reverse_token_dict.get, titles[index])

In [ ]:
'''
training_titles = data['training_titles']
target_titles = data['target_titles']

index = 20000
print training_titles[index]
print target_titles[index]
'''

### validate the DataGenerator with dual outputs

In [ ]:
from data_preprocess import TOKEN_DICT, _GO, _EOS
from data import DataGenerator, process_batch

In [ ]:
pickle_file = 'scramble_titles_data.pkl'
batch_size = 32
pickle_file_path = os.path.join(os.path.expanduser("~"), pickle_file)

## the dual_output test
#dataGen = DataGenerator(pickle_file_path)
dataGen = DataGenerator(pickle_file_path, True)
batches = dataGen.generate_sequence(batch_size)
training_batch, target_batch = next(batches)
  

In [ ]:
training_batch

In [ ]:
target_batch

In [ ]:
encoder_inputs_, encoder_inputs_length = process_batch([sequence + [TOKEN_DICT[_EOS]] for sequence in training_batch])
decoder_inputs_, decoder_inputs_length = process_batch([[TOKEN_DICT[_GO]] + sequence for sequence in target_batch])


In [ ]:
encoder_inputs_

In [ ]:
decoder_inputs_